# ETL archivos Yelp
El objetivo es poder entender la naturaleza, formato y estructura que conteinen los archivos de Yelp para luego realizar el proceso de ETL.

* business.pkl
* review.json
* user.parquet
* checkin.json
* tip.json


In [140]:
import pandas as pd
import numpy as np
import requests
import glob
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
import pyarrow.parquet as pq
import json
import pickle

## checkin

In [141]:
def abrir_Archivo_json(archivo):
    merged_data = []  # Lista para almacenar los objetos JSON combinados

    with open(archivo) as file:
        for line in file:
            try:
                obj = json.loads(line)
                merged_data.append(obj)
            except json.JSONDecodeError as e:
                print(f"Error al decodificar JSON en el archivo {archivo}: {str(e)}")

    df = pd.DataFrame(merged_data)  # Crear DataFrame a partir de los objetos JSON
    return df


In [142]:
archivo = 'data/Yelp/checkin.json' 
df_checkin_yelp = abrir_Archivo_json(archivo)

In [143]:
#Revisamos si tiene nulos
df_checkin_yelp.isnull().sum()

business_id    0
date           0
dtype: int64

In [144]:
#Revisamos si hay duplicados
df_checkin_yelp.duplicated().sum()

0

Desanidado de Fecha y Hora

In [145]:
fecha = []
for _, row in df_checkin_yelp.iterrows():
    business_id = row['business_id']
    dates = row['date'].split(', ')
    for date in dates:
        time, date = date.split(' ')
        fecha.append([business_id, time, date])

df_desanidado = pd.DataFrame(fecha,columns =['business_id','hour','date'])

In [146]:
df_desanidado

,business_id,hour,date
0,---kPU91CF4Lq2-WlRu9Lw,2020-03-13,21:10:56
1,---kPU91CF4Lq2-WlRu9Lw,2020-06-02,22:18:06
2,---kPU91CF4Lq2-WlRu9Lw,2020-07-24,22:42:27
3,---kPU91CF4Lq2-WlRu9Lw,2020-10-24,21:36:13
4,---kPU91CF4Lq2-WlRu9Lw,2020-12-09,21:23:33
...,...,...,...
13356870,zzu6_r3DxBJuXcjnOYVdTw,2013-12-11,00:52:49
13356871,zzu6_r3DxBJuXcjnOYVdTw,2013-12-13,00:58:14
13356872,zzw66H6hVjXQEt0Js3Mo4A,2016-12-03,23:33:26
13356873,zzw66H6hVjXQEt0Js3Mo4A,2018-12-02,19:08:45


In [147]:
#Se almacena en un csv
df_desanidado.to_csv('data/Yelp/checkin.csv',index = False)

## user.parquet

## business.pkl

In [148]:
#se lee el archivo
df_business= pd.read_pickle('data/Yelp/business.pkl')

In [149]:
df_business

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,IN,T6J 5H2,53.468419,-113.492054,3.0,13,1,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","Nail Salons, Beauty & Spas","{'Monday': '10:0-19:30', 'Tuesday': '10:0-19:3...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,DE,37204,36.115118,-86.766925,4.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Pets, Nurseries & Gardening, Pet Stores, Hobby...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,AB,46250,39.908707,-86.065088,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Shopping, Jewelry, Piercing, Toy Stores, Beaut...",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,AB,62025,38.782351,-89.950558,4.0,24,1,"{'BusinessParking': '{'garage': False, 'street...","Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [150]:
#Se agregan nuevas columnas en el Dataframe Vacias
df_business['NAME']=None
df_business['REVIEW_COUNT']=None
df_business['POSTAL_CODE']=None
df_business['CITY']=None
df_business['STATE']=None
df_business['BUSINESS_ID']=None
df_business['ADDRESS']=None
df_business['LATITUDE']=None
df_business['LONGITUDE']=None
df_business['STARS']=None
df_business['IS_OPEN']=None
df_business['ATTRIBUTES']=None
df_business['CATEGORIES']=None
df_business['HOURS']=None

In [151]:
#Se actualiza la columna NAME del Dataframe donde se fusionan las cadenas de texto, eliminando los caracteres no Alfabeticos
for index,i in enumerate(df_business.name.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['NAME'][index]=''.join(arr)


In [152]:

# Configurar pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)


In [153]:
#Se actualiza la columna CITY del Dataframe donde se fusionan las cadenas de texto, eliminando los caracteres no Alfabeticos
for index,i in enumerate(df_business.city.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['CITY'][index]=''.join(arr)

In [154]:
#Se actualiza la columna STATE del Dataframe donde se fusionan las cadenas de texto, eliminando los caracteres no Alfabeticos
for index,i in enumerate(df_business.state.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['STATE'][index]=''.join(arr)


In [155]:
#Se actualiza la columna REVIEW_COUNT del Dataframe donde se fusionan las cadenas de texto, eliminando los caracteres no Alfabeticos
for index,i in enumerate(df_business.review_count.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['REVIEW_COUNT'][index]=''.join(arr)


In [156]:
#Se actualiza la columna BUSINESS_ID del Dataframe donde se fusionan las cadenas de texto, eliminando los caracteres no Alfabeticos
for index,i in enumerate(df_business.business_id.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['BUSINESS_ID'][index]=''.join(arr)


In [157]:
#Se actualiza la columna ADDRESS del Dataframe donde se fusionan las cadenas de texto, eliminando los caracteres no Alfabeticos
for index,i in enumerate(df_business.address.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['ADDRESS'][index]=''.join(arr)


In [158]:
#Se actualiza la columna POSTAL_CODE del Dataframe donde se fusionan las cadenas de texto, eliminando los caracteres no Alfabeticos
for index,i in enumerate(df_business.postal_code.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['POSTAL_CODE'][index]=''.join(arr)

In [159]:
#Se filtran los elementos numéricos mayores que 1 en cada valor de la columna 'latitude' del DataFrame 'df_business'.
# A continuación, se asigna el primer elemento filtrado a la columna 'LATITUDE' en el DataFrame 'df_business' en la fila
#correspondiente al índice actual.
for index,i in enumerate(df_business.latitude.values):
    arr=[]
    for e in i:
       if e>1:
         arr.append(e)
    df_business['LATITUDE'][index]=arr[0]


In [160]:
#Se filtran los elementos numéricos menores que -1 en cada valor de la columna 'longitude' del DataFrame 'df_business'.
#Luego, se asigna el primer elemento filtrado a la columna 'LONGITUDE' en el DataFrame 'df_business' en la fila correspondiente al
# índice actual.
for index,i in enumerate(df_business.longitude.values):
    arr=[]
    for e in i:
        if e<-1:
            arr.append(e)
    df_business['LONGITUDE'][index]=arr[0]

In [161]:
#Se filtran los elementos numéricos mayores que 0.1 en cada valor de la columna 'stars' del DataFrame 'df_business'.
# Después, se asigna el primer elemento filtrado a la columna 'STARS' en el DataFrame 'df_business' en la fila correspondiente 
#al índice actual.
for index,i in enumerate(df_business.stars.values):
    arr=[]
    for e in i:
       if e>0.1:
         arr.append(e)
    df_business['STARS'][index]=arr[0]


In [162]:
#Se filtran los elementos numéricos mayores o iguales a 0 en cada valor de la columna 'is_open' del DataFrame 'df_business'. 
#A continuación, se asigna el primer elemento filtrado a la columna 'IS_OPEN' en el DataFrame 'df_business' en la fila correspondiente al índice actual

for index,i in enumerate(df_business.is_open.values):
    arr=[]
    for e in i:
       if e >=0:
         arr.append(e)
    df_business['IS_OPEN'][index]=arr[0]

In [163]:
#Este código busca y guarda el primer diccionario encontrado en la columna 'attributes' del dataframe 'df_business' en la columna 'ATTRIBUTES'.
for index,i in enumerate(df_business.attributes.values):
    arr=[]
    for e in i:
        if isinstance(e,dict):
         arr.append(e)
    if len(arr)>0:
     df_business['ATTRIBUTES'][index]=arr[0]


In [164]:
#Este código filtra los elementos de tipo cadena en la columna 'categories' del dataframe 'df_business'. 
#Luego, fusiona todas las cadenas filtradas en una sola cadena y la asigna a la columna 'CATEGORIES' en el dataframe 'df_business'.
for index,i in enumerate(df_business.categories.values):
    arr=[]
    for e in i:
        if isinstance(e,str):
         arr.append(e)
    df_business['CATEGORIES'][index]=''.join(arr)

In [165]:
#En este código, se recorren los valores de la columna 'hours' en el dataframe 'df_business' y se filtran los elementos que son diccionarios. 
#Luego, se asigna el primer diccionario encontrado a la columna 'HOURS' en el dataframe 'df_business' en la fila correspondiente.
#En resumen, el código extrae y guarda el primer diccionario encontrado en la columna 'hours' del dataframe 'df_business' en la columna 'HOURS'.
for index,i in enumerate(df_business.hours.values):
    arr=[]
    for e in i:
        if isinstance(e,dict):
         arr.append(e)
    if len(arr)>0:     
     df_business['HOURS'][index]=arr[0]


In [166]:
df_business.sample(10)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,NAME,REVIEW_COUNT,POSTAL_CODE,CITY,STATE,BUSINESS_ID,ADDRESS,LATITUDE,LONGITUDE,STARS,IS_OPEN,ATTRIBUTES,CATEGORIES,HOURS
100227,9D0aKRGsutg8S0ClIanmrA,Rival Bros Coffee,2400 Lombard St,Philadelphia,PA,19146,39.946558,-75.180536,4.5,143,1,"{'BikeParking': 'True', 'HasTV': 'False', 'Res...","Restaurants, Coffee Roasteries, Food, Cafes, C...","{'Monday': '7:0-14:0', 'Tuesday': '7:0-14:0', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rival Bros Coffee,,19146,Philadelphia,PA,9D0aKRGsutg8S0ClIanmrA,2400 Lombard St,39.946558,-75.180536,4.5,1,"{'BikeParking': 'True', 'HasTV': 'False', 'Res...","Restaurants, Coffee Roasteries, Food, Cafes, C...","{'Monday': '7:0-14:0', 'Tuesday': '7:0-14:0', ..."
145831,0BnyboiYrgPsSTL4Lf_uLw,I Scream Maplewood,7326B Manchester Rd,Maplewood,CA,63143,38.612824,-90.318206,4.0,5,0,"{'Caters': 'False', 'DogsAllowed': 'True', 'Re...","Food Trucks, Food, Desserts, Ice Cream & Froze...","{'Monday': '14:0-22:0', 'Tuesday': '14:0-22:0'...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I Scream Maplewood,,63143,Maplewood,CA,0BnyboiYrgPsSTL4Lf_uLw,7326B Manchester Rd,38.612824,-90.318206,4.0,0,"{'Caters': 'False', 'DogsAllowed': 'True', 'Re...","Food Trucks, Food, Desserts, Ice Cream & Froze...","{'Monday': '14:0-22:0', 'Tuesday': '14:0-22:0'..."
14899,GlYN-RJyL_YwxIqsSDo1ag,Mother Hubbard's Café,14 W Grant Rd,Tucson,NV,85705,32.251045,-110.972895,3.5,250,0,"{'RestaurantsPriceRange2': '1', 'Caters': 'Fal...","Diners, American (Traditional), Restaurants, B...","{'Wednesday': '7:0-13:0', 'Thursday': '7:0-13:...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mother Hubbard's Café,,85705,Tucson,NV,GlYN-RJyL_YwxIqsSDo1ag,14 W Grant Rd,32.251045,-110.972895,3.5,0,"{'RestaurantsPriceRange2': '1', 'Caters': 'Fal...","Diners, American (Traditional), Restaurants, B...","{'Wednesday': '7:0-13:0', 'Thursday': '7:0-13:..."
133229,b81o5QrzPKjq8P5zlNYU0g,Tennessee Nail Spa,"202 N Anderson Ln, Ste 103",Hendersonville,PA,37075,36.320703,-86.596944,3.5,28,1,"{'BikeParking': 'True', 'BusinessParking': '{'...","Nail Salons, Hair Salons, Skin Care, Beauty & ...","{'Monday': '9:30-19:30', 'Tuesday': '9:30-19:3...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tennessee Nail Spa,,37075,Hendersonville,PA,b81o5QrzPKjq8P5zlNYU0g,"202 N Anderson Ln, Ste 103",36.320703,-86.596944,3.5,1,"{'BikeParking': 'True', 'BusinessParking': '{'...","Nail Salons, Hair Salons, Skin Care, Beauty & ...","{'Monday': '9:30-19:30', 'Tuesday': '9:30-19:3..."
7247,GnXLe-SHVmlvx8vcU-I68w,Patelka Dental,"8332 Bustleton Ave, Unit C",Philadelphia,PA,19152,40.06877,-75.051828,1.0,10,1,{'ByAppointmentOnly': 'False'},"General Dentistry, Health & Medical, Dentists,...","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Patelka Dental,,19152,Philadelphia,PA,GnXLe-SHVmlvx8vcU-I68w,"8332 Bustleton Ave, Unit C",40.06877,-75.051828,1.0,1,{'ByAppointmentOnly': 'False'},"General Dentistry, Health & Medical, Dentists,...","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
12178,GYBeFvSP4qASUKP8lMPX8g,Aunt Berta's Kitchen Too,311 N White Horse Pike,Lindenwold,FL,08021,39.822504,-74.995873,4.0,13,1,"{'Alcohol': 'u'none'', 'BusinessAcceptsCreditC...","Restaurants, Soul Food","{'Wednesday': '11:0-19:30', 'Thursday': '11:0-...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aunt Berta's Kitchen Too,,08021,Lindenwold,FL,GYBeFvSP4qASUKP8lMPX8g,311 N White Horse Pike,39.822504,-74.995873,4.0,1,"{'Alcohol': 'u'none'', 'BusinessAcceptsCreditC...","Restaurants, Soul Food","{'Wednesday': '11:0-19:30', 'Thursday': '11:0-..."
117543,Mro39taLejqtvsitvMCfGg,Mercy Eye Care,34 Hampton Village Plz,Saint Louis,FL,63109,38.592131,-90.292306,3.5,

In [167]:
df_business.to_csv('data/Yelp/business.csv',index = False)

In [168]:
dtypes = {'col14': str, 'col15': str, 'col16': str, 'col17': str, 'col18': str, 'col19': str, 'col25': str, 'col26': str, 'col27': str}
df_business = pd.read_csv('data/Yelp/business.csv', dtype=dtypes)

C:\Users\nadir\AppData\Local\Temp\ipykernel_23500\2900228667.py:2: DtypeWarning: Columns (14,15,16,17,18,19,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_business = pd.read_csv('data/Yelp/business.csv', dtype=dtypes)


In [169]:
#Filtramos el dataframe solo a las columnas nuevas generadas con los bucles, con los nombres iniciales


In [170]:
# Obtén la lista de todas las columnas
all_columns = df_business.columns

# Itera sobre todas las columnas y aplica el método .loc
for col in all_columns:
    df_business.loc[:, col] = df_business[col]


In [171]:
df_business = df_business[['BUSINESS_ID', 'NAME', 'REVIEW_COUNT', 'CITY', 'STATE', 'ADDRESS', 'POSTAL_CODE', 'LATITUDE', 'LONGITUDE', 'STARS', 'IS_OPEN', 'ATTRIBUTES', 'CATEGORIES', 'HOURS']]

df_business.loc[:, 'business_id'] = df_business['BUSINESS_ID']
df_business.loc[:, 'name'] = df_business['NAME']
df_business.loc[:, 'address'] = df_business['ADDRESS']
df_business.loc[:, 'city'] = df_business['CITY']
df_business.loc[:, 'state'] = df_business['STATE']
df_business.loc[:, 'postal_code'] = df_business['POSTAL_CODE']
df_business.loc[:, 'latitude'] = df_business['LATITUDE']
df_business.loc[:, 'longitude'] = df_business['LONGITUDE']
df_business.loc[:, 'stars'] = df_business['STARS']
df_business.loc[:, 'review_count'] = df_business['REVIEW_COUNT']
df_business.loc[:, 'is_open'] = df_business['IS_OPEN']
df_business.loc[:, 'attributes'] = df_business['ATTRIBUTES']
df_business.loc[:, 'categories'] = df_business['CATEGORIES']
df_business.loc[:, 'hours'] = df_business['HOURS']

df_business = df_business[['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'attributes', 'categories', 'hours']]


In [172]:
df_business

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,NaN,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,NaN,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,NaN,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,NaN,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,NaN,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,IN,T6J 5H2,53.468419,-113.492054,3.0,NaN,1,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","Nail Salons, Beauty & Spas","{'Monday': '10:0-19:30', 'Tuesday': '10:0-19:3..."
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,DE,37204,36.115118,-86.766925,4.0,NaN,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Pets, Nurseries & Gardening, Pet Stores, Hobby...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3..."
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,AB,46250,39.908707,-86.065088,3.5,NaN,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Shopping, Jewelry, Piercing, Toy Stores, Beaut...",NaN
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,AB,62025,38.782351,-89.950558,4.0,NaN,1,"{'BusinessParking': ""{'garage': False, 'street...","Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ..."


In [176]:
filtro = ['Restaurant']

# Filtrar el DataFrame solo para las filas donde 'categories' no es NaN
df_filtered = df_business.dropna(subset=['categories'])

# Luego, aplicar el filtro
restaurante = df_filtered[df_filtered['categories'].str.contains('|'.join(filtro), regex=True)]


In [178]:
restaurante

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,NaN,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,AZ,37015,36.269593,-87.058943,2.0,NaN,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,TN,63123,38.565165,-90.321087,3.0,NaN,0,"{'Caters': 'True', 'Alcohol': ""u'full_bar'"", '...","Pubs, Restaurants, Italian, Bars, American (Tr...",NaN
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,MO,37207,36.208102,-86.768170,1.5,NaN,1,"{'RestaurantsAttire': ""'casual'"", 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '..."
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,MO,33602,27.955269,-82.456320,4.0,NaN,1,"{'Alcohol': ""'none'"", 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150325,l9eLGG9ZKpLJzboZq-9LRQ,Wawa,19 N Bishop Ave,Clifton Heights,TN,19018,39.925656,-75.310344,3.0,NaN,1,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Restaurants, Sandwiches, Convenience Stores, C...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
150327,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,1181 N Milwaukee St,Boise,ID,83704,43.615401,-116.284689,4.0,NaN,1,"{'WiFi': ""'free'"", 'RestaurantsGoodForGroups':...","Cafes, Juice Bars & Smoothies, Coffee & Tea, R...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-17:0', '..."
150336,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,1108 S 9th St,Philadelphia,MO,19147,39.935982,-75.158665,4.5,NaN,1,"{'WheelchairAccessible': 'False', 'Restaurants...","Restaurants, Mexican","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
150339,2O2K6SXPWv56amqxCECd4w,The Plum Pit,4405 Pennell Rd,Aston,PA,19014,39.856185,-75.427725,4.5,NaN,1,"{'RestaurantsDelivery': 'False', 'BusinessAcce...","Restaurants, Comfort Food, Food, Food Trucks, ...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."


In [179]:
restaurante.to_csv('data/Yelp/restaurante.csv')

In [ ]:
df_business.to_csv('data/Yelp/business.csv',index = False)

## tip.json

In [ ]:
json_objects=[]

with open('data/Yelp/tip.json', 'r',encoding='utf-8') as f:
    for line in f:
        json_objects.append(json.loads(line))
df_tip = pd.DataFrame(json_objects)

In [ ]:
df_tip

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0
...,...,...,...,...,...
908910,eYodOTF8pkqKPzHkcxZs-Q,3lHTewuKFt5IImbXJoFeDQ,Disappointed in one of your managers.,2021-09-11 19:18:57,0
908911,1uxtQAuJ2T5Xwa_wp7kUnA,OaGf0Dp56ARhQwIDT90w_g,Great food and service.,2021-10-30 11:54:36,0
908912,v48Spe6WEpqehsF2xQADpg,hYnMeAO77RGyTtIzUSKYzQ,Love their Cubans!!,2021-11-05 13:18:56,0
908913,ckqKGM2hl7I9Chp5IpAhkw,s2eyoTuJrcP7I_XyjdhUHQ,Great pizza great price,2021-11-20 16:11:44,0


NameError: name 'BusinessYelp' is not defined

## review.json

In [184]:
def abrir_Archivo_json(archivo):
    merged_data = []  # Lista para almacenar los objetos JSON combinados

    with open(archivo,encoding = 'utf-8') as file:
        for line in file:
            try:
                obj = json.loads(line)
                merged_data.append(obj)
            except json.JSONDecodeError as e:
                print(f"Error al decodificar JSON en el archivo {archivo}: {str(e)}")

    df = pd.DataFrame(merged_data)  # Crear DataFrame a partir de los objetos JSON
    return df

In [186]:
archivo = ('data/Yelp/review.json')
review=abrir_Archivo_json(archivo)

MemoryError: Unable to allocate 480. MiB for an array with shape (6990280, 9) and data type object